In [1]:
!pip install -r /workspaces/Naive_Bayes/requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Carga del conjunto de datos
import pandas as pd

# dataset
data = "/workspaces/Naive_Bayes/data/raw/playstore_reviews.csv"

# Cargar el dataset
df = pd.read_csv(data)

# Mostrar las primeras filas
print("Primeras filas del dataset:")
display(df.head())


Primeras filas del dataset:


,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [3]:
# Exploracíon
# Dimensiones del dataset
print("\nDimensiones del dataset:", df.shape)

# Información general
print("\nInformación del dataset:")
df.info()

# Valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())



Dimensiones del dataset: (891, 3)

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB

Valores nulos por columna:
package_name    0
review          0
polarity        0
dtype: int64


In [4]:
# Eliminación de duplicados y columnas innecesarias

# Eliminación de duplicados
df.drop_duplicates(inplace=True)
print("\nDimensiones después de eliminar duplicados:", df.shape)

# Eliminación de la columna package_name (no es útil)
df.drop(columns=["package_name"], inplace=True)

# Valores únicos de la variable de salida
print("\nValores únicos en polarity:", df["polarity"].unique())



Dimensiones después de eliminar duplicados: (891, 3)

Valores únicos en polarity: [0 1]


In [5]:
# Procesamiento del texto

# Convertimos el texto a minúsculas y eliminamos espacios en blanco
df["review"] = df["review"].str.strip().str.lower()

# Verificamos los cambios
print("\nPrimeras filas después de limpiar el texto:")
display(df.head())



Primeras filas después de limpiar el texto:


,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0


In [6]:
# División en Train y Test
from sklearn.model_selection import train_test_split

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["polarity"], test_size=0.2, random_state=42)

print("\nTamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))



Tamaño del conjunto de entrenamiento: 712
Tamaño del conjunto de prueba: 179


In [7]:
# Vectorización del Texto
from sklearn.feature_extraction.text import CountVectorizer

# Transformación de texto en una matriz de recuento de palabras
vec_model = CountVectorizer(stop_words="english")

# Aplicamos la transformación a los datos de entrenamiento
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

print("\nDimensiones después de la vectorización:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)



Dimensiones después de la vectorización:
X_train: (712, 3310)
X_test: (179, 3310)


In [8]:
# Entrenamiento de Naive Bayes
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

# Evaluamos los tres modelos de Naive Bayes
models = {
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB()
}

print("\nEvaluando modelos Naive Bayes:")
for name, model in models.items():
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"{name} Accuracy: {score:.4f}")



Evaluando modelos Naive Bayes:
GaussianNB Accuracy: 0.8045
MultinomialNB Accuracy: 0.8156
BernoulliNB Accuracy: 0.7709


In [9]:
# Selección del mejor modelo
# Selección del mejor modelo basado en los resultados
best_model = MultinomialNB()
best_model.fit(X_train, y_train)

print("\nModelo seleccionado: MultinomialNB")



Modelo seleccionado: MultinomialNB


In [10]:
# Optimización con Random Forest
from sklearn.ensemble import RandomForestClassifier

print("\nEntrenando modelo Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_test, y_test)

print(f"Random Forest Accuracy: {rf_score:.4f}")



Entrenando modelo Random Forest...
Random Forest Accuracy: 0.7989


In [11]:
# Guardado del modelo
import joblib

# Guardamos el mejor modelo y el vectorizador
joblib.dump(best_model, "naive_bayes_model.pkl")
joblib.dump(vec_model, "vectorizer.pkl")

print("\nModelo Naive Bayes y vectorizador guardados correctamente.")



Modelo Naive Bayes y vectorizador guardados correctamente.


In [12]:
# Exploración de otros modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

print("\nEntrenando modelos adicionales...")

# Regresión Logística
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
log_reg_score = log_reg.score(X_test, y_test)
print(f"Logistic Regression Accuracy: {log_reg_score:.4f}")

# Árboles de Decisión
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
dt_score = dt_model.score(X_test, y_test)
print(f"Decision Tree Accuracy: {dt_score:.4f}")

# Comparación de Modelos
print("\nComparación de Modelos:")
print(f"Naive Bayes (MultinomialNB): {best_model.score(X_test, y_test):.4f}")
print(f"Random Forest: {rf_score:.4f}")
print(f"Regresión Logística: {log_reg_score:.4f}")
print(f"Árbol de Decisión: {dt_score:.4f}")

# Selección del mejor modelo final
best_model_name = "Random Forest" if rf_score > best_model.score(X_test, y_test) else "Naive Bayes"
print("\nEl mejor modelo es:", best_model_name)



Entrenando modelos adicionales...
Logistic Regression Accuracy: 0.8324
Decision Tree Accuracy: 0.7151

Comparación de Modelos:
Naive Bayes (MultinomialNB): 0.8156
Random Forest: 0.7989
Regresión Logística: 0.8324
Árbol de Decisión: 0.7151

El mejor modelo es: Naive Bayes
